<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [2]:
#Импорт необходимых библиотек
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
from plotly import express as px

In [4]:
#Устанавливаем подключение к базе данных
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
query_3_1 = f'''
select 
count(id) --cчитаем количество идентификаторов вакансий в таблице
from
vacancies
'''

In [6]:
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3851344990.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
query_3_2 = f'''
select 
count(*) --cчитаем количество записей в таблице работодателей
from
employers
'''

In [8]:
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3588252056.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
query_3_3 = f'''
select 
count(id)--cчитаем количество идентификаторов в таблице регионов
from
areas
'''

In [10]:
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\2018224393.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
query_3_4 = f'''
select 
    count(*) 
from
    industries
'''

In [12]:
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\2172479966.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,count
0,294


***

В блоке предварительного анализа данных мы получили информацию о количественных показателях различных признаков. 
Мы получили представление о размерности табиц в базе данных, в дальнейшем это может помочь с выбором правильного типа данных для отображения. 

Ниже представлен график с топ-20 отраслей по количеству вакансий.Мы видим, что ИТ-сфера опережает остальные отрасли с большим запасом. Общее количество вакансий в графике выше, чем количество вакансий в таблице, так как у многих работодателей указаны более одной сферы деятельности, и их вакансии попадают в каждую отрасль.


In [13]:
#создаем запрос
query_ind = '''
select
    i.name,
    count(v.id)
from
-- поскольку прямой связи между таблицами вакансий и отраслей нет, используем связь через промежуточную таблицу employers_industries
    vacancies v join employers_industries e on v.employer_id = e.employer_id join industries i on e.industry_id = i.id 
group by 1
order by 2 desc limit 20
'''
# Выполняем запрос, получаем датафрейм
df = pd.read_sql_query(query_ind, connection)
# Используем сегментную диаграмму для большей наглядности
fig = px.pie(df, names='name', values= 'count') 
fig.show()


C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3389707893.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_ind, connection)


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [14]:
query_4_1 = '''
select
    count(v.id),
    a.name
from
    vacancies v join areas a on v.area_id = a.id -- объединяем таблицы с вакансиями и регионами размещения по полю с идентификатором региона
group by a.id
order by 1 desc
'''

In [15]:
# Получим датафрейм с количеством вакансий для каждого региона
df = pd.read_sql_query(query_4_1, connection)
# Покажем топ-5 из них
display(df.head(5))
# Для большей наглядности выведем столбчатую диаграмму
fig = px.bar(df[:5], x='name', y='count')
fig.show()

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3110121951.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [16]:
query_4_2 = '''
 -- выбираем компании, указавшие зарплату
 select
    e.name,
    count(v.id)
from
    vacancies v join employers e on v.employer_id = e.id 
where v.salary_from is not null or v.salary_to is not null 
group by 1
union 
-- получим общее количество вакансий с указанной з\п
select
    'total',
    count(v.id)
from
    vacancies v 
where v.salary_from is not null or v.salary_to is not null 
order by 2 desc -- сортируем вывод по убыванию
'''

In [17]:
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\2168503709.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,total,24073
1,Яндекс,1495
2,Ростелеком,392
3,Тензор,155
4,Спецремонт,151
...,...,...
10486,Ортоника,1
10487,ТЭМПО-Логистик,1
10488,Белов Андрей Валентинович,1
10489,Теорема Телеком,1


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [18]:
query_4_3 = '''
select
    round(avg(v.salary_from)) lower,
    round(avg(v.salary_to)) upper
from
    vacancies v 
'''

In [19]:
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\2222372203.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,lower,upper
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [20]:
query_4_4 = '''
select
    count(v.id),
    schedule || ' ' || employment job
from
    vacancies v
group by 2
order by 1 desc
'''

In [21]:
df = pd.read_sql_query(query_4_4, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\2981602351.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count,job
0,35367,Полный день Полная занятость
1,7802,Удаленная работа Полная занятость
2,1593,Гибкий график Полная занятость
3,1312,Удаленная работа Частичная занятость
4,940,Сменный график Полная занятость
5,569,Полный день Стажировка
6,367,Вахтовый метод Полная занятость
7,347,Полный день Частичная занятость
8,312,Гибкий график Частичная занятость
9,141,Полный день Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [22]:
query_4_5 = '''
select
    count(v.id),
    v.experience
from
    vacancies v
group by 2
order by 1
'''

In [23]:
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\2601166264.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

В первой части мы получили довольно очевидный результат: наибольшее количество вакансий представлены в крупных и развитых городах. Чем больше население города, чем выше уровень его развития - тем больше сотрудников требуется.  

Во втором пункте становится видно, что больше зарплата не указана больше, чем в половине случаев. Из 49 000 вакансий, зарплату показывают только в 24 000 случаев (10 490 из 23 501 работодателей). 
Из третьего пунка понятно, что зарплату в вакансии указывают в основном тогда, когда она превышает среднее значение (по данным исследований средняя зарплата в РФ в I квартале 2024г - 68 000 рублей, медианная около 50 000 рублей - данные получены вручную). 

В результатах четвертого запроса видно, что "традиционный" график работы "Полный день\Полная занятость" все еще остается наиболее востребованным на рынке труда. При этом заметно увеличилось количество вакансий предполагающих удаленную работу (2 и 4 место по количеству вакансий) или гибкий график (3 место). 

По пятому пункту можно понять, что компании ищут, в основном, начинающих специалистов и специалистов среднего уровня, с небольшим практическим опытом. Меньше всего вакансий для максимально опытных работников. 
Можно выдвинуть три гипотезы:
- Опытные сотрудники не склонны к смене места работы.
- Количество высокоответственных должностей, требущих такого уровня опыта наименьшее по рынку труда.
- Сотрудников на подобные позиции ищут вне интернет-ресурсов.

Подтвердить или опровергнуть эти гипотезы можно в процессе дальнейшего анализа.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [24]:
query_5_1 = '''
select
    count(v.id),
    e.name
    
from
    vacancies v join employers e on v.employer_id = e.id
group by 2
order by 1 desc limit 5
'''

In [25]:
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\1792814825.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
query_5_2 = '''
select
    a.name,
    count(v.id) amnt_vacancies,
    count(e.id) amnt_employers
from
    areas a left join vacancies v on v.area_id = a.id -- используем левое соединение, поскольку нужны все регионы, в том числе и те, в которых вакансий нет
    left join employers e on e.area = a.id
where v.id is null 
group by 1
order by 2, 3 desc
'''

In [27]:
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3341481329.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,amnt_vacancies,amnt_employers
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
query_5_3 = '''
select
    e.name,
    count(distinct v.area_id)
from
    employers e join vacancies v on e.id = v.employer_id
group by 1
order by 2 desc
    '''

In [29]:
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\2398953733.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
query_5_4 = '''
select
    count(e.id)
from
    employers e left join employers_industries i on e.id = i.employer_id
where i.industry_id is null
'''

In [31]:
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3114411492.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
query_5_5 = '''
select
    e.name,
    count(i.industry_id)
from
    employers e join employers_industries i on e.id = i.employer_id
group by e.name
having count(i.industry_id) = 4
order by 1 limit 1 offset 2
'''

In [33]:
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\1650419958.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
query_5_6 = '''
select
    count(e.employer_id)
from
    employers_industries e join industries i on e.industry_id = i.id
where i.name='Разработка программного обеспечения'
    
'''

In [35]:
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\1169039133.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
# Получаем данные из статьи на Википедии
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
# С помощью инструмента разработчика в браузере выяснили, какими тегами оформлена нужная информация, находим нужную таблицу, из таблицы получаем данные
table_city = BeautifulSoup(requests.get(url).text, 'html.parser').find('table', class_='standard sortable').find_all('a')
city_list = []
# В список попали служебные данные из таблицы, фильтруем в цикле.
for now in table_city:
    if now.text.replace('-', '').isalpha() or now.text.replace(' ', '').isalpha():
        city_list.append(now.text)
city_list

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [37]:
query_5_7 = f'''
select
    a.name,
    count(v.id)
from
    vacancies v join employers e on e.id = v.employer_id 
    join areas a on v.area_id = a.id
where e.name='Яндекс' and a.name in {tuple(city_list)}
group by a.name
union
select
    'total', count(v.id)
from
    vacancies v join employers e on e.id = v.employer_id
    join areas a on v.area_id = a.id
    where e.name='Яндекс' and a.name in {tuple(city_list)}
'''

In [38]:
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3129374555.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,total,485
1,Волгоград,24
2,Воронеж,32
3,Екатеринбург,39
4,Казань,25
5,Краснодар,30
6,Красноярск,23
7,Москва,54
8,Нижний Новгород,36
9,Новосибирск,35


***

Из проведенных запросов видно, что наибольшее количество вакансий представлено в сфере IT и банковском секторе. Также видно, что при размещении вакансии работодатель предпочитает указывать конкретный регион. Для вакансий, подразумевающих удаленный формат, вакансии размещаются в нескольких регионах. 

При этом больше половины работодателей не указывают сферу деятельности компании. Среди указавших около 50% казывают одной из сфер деятельности разработку ПО. 

Мы наиболее внимательно изучили вакансии Яндекса, который размещает свои вакансии в максимальном количестве регионов. Также вакансии Яндекса затрагивают все города-миллионники РФ. 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [39]:
query_6_1 = '''
select
    count(v.id)
from
    vacancies v
where lower(v.name) like '%%data%%' or lower(v.name) like '%%данн%%'
'''

In [40]:
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\485020324.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [41]:
query_6_2 = '''
select
    count(v.id)
from
    vacancies v
where 
(lower(v.name) like '%data scientist%' 
or lower(v.name) like '%data science%' 
or lower(v.name) like '%исследователь данных%'
or v.name like '%ML%' and v.name not like '%HTML%'
or lower(v.name) like '%machine learning%' 
or lower(v.name) like '%машинн%обучен%')
and
(lower(v.name) like '%junior%' 
or lower(v.experience) = 'нет опыта'
or lower(v.employment) = 'стажировка')
'''

In [42]:
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3641650667.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [43]:
query_6_3 = '''
select
    count(v.id)
from
    vacancies v
where 
(lower(v.name) like '%data scientist%' 
or lower(v.name) like '%data science%' 
or lower(v.name) like '%исследователь данных%'
or lower(v.name) like '%ml%' and v.name not like '%HTML%' and v.name not like '%QML%'
or lower(v.name) like '%machine learning%' 
or lower(v.name) like '%машинн%обучен%')
and
(v.key_skills like '%SQL%' or v.key_skills like '%postgres%')
'''

In [44]:
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\1159587760.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [45]:
query_6_4 = '''
select
    count(v.id)
from
    vacancies v
where 
(lower(v.name) like '%data scientist%' 
or lower(v.name) like '%data science%' 
or lower(v.name) like '%исследователь данных%'
or lower(v.name) like '%ml%' and v.name not like '%HTML%' and v.name not like '%QML%'
or lower(v.name) like '%machine learning%' 
or lower(v.name) like '%машинн%обучен%')
and
(v.key_skills like '%Python%')
'''

In [46]:
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\972386973.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [47]:
query_6_5 = '''
select
-- считаем среднее количество ключевых навыков
    round(avg(length(key_skills) -- длина строки с навыками

     -length(replace(key_skills, CHR(9), ''))+1), 2) -- длина строки с навыками без разделителей (символ табуляции)
from
    vacancies v
where 
(lower(v.name) like '%data scientist%' 
or lower(v.name) like '%data science%' 
or lower(v.name) like '%исследователь данных%'
or v.name like '%ML%' and v.name not like '%HTML%'
or lower(v.name) like '%machine learning%' 
or lower(v.name) like '%машинн%обучен%')'''

In [48]:
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\390248827.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [49]:
query_6_6 = '''
select
    experience,
    round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))/2)) -- считаем среднюю зарплату, используем данные, указанные в вакансии
from
    vacancies v
where 
(v.salary_from is not null or v.salary_to is not null) -- игнорируем пустые значения
and
-- условия области деятельности:
(lower(v.name) like '%data scientist%' 
or lower(v.name) like '%data science%' 
or lower(v.name) like '%исследователь данных%'
or v.name like '%ML%' and v.name not like '%HTML%'
or lower(v.name) like '%machine learning%' 
or lower(v.name) like '%машинн%обучен%')
group by experience
'''

In [50]:
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\3561045109.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

В первом запросе получена 1771 вакансия, при этом для начинающего специалиста по данным (второй запрос) подходит всего 51 вакансия. Компании возможно,опасаются брать совсем начинащих сотрудников, либо в описании вакансий указывают завышенные ожидания опыта. Выяснить это можно при дальнейшем анализе ключевых навыков.

В отношении зарплаты из шестого запроса видно, что оплата в области работы с данными выше средней зарплаты по РФ даже для соискателей без опыта

В среднем в вакансиях, имеющих отношение к данным, указаны 6,41 ключевых навыков. При этом в относительно небольшом количестве случаев ключевыми навыками названы Python и SQL\Postgres (357 и 229 соответственно). Можно сделать вывод, что для успешной работы кандидат должен обладать бОльшим количеством навыков. 

In [51]:
# Запрос дл получения полного списка навыков, встречающихся в данных
query_skills = '''
select
   string_agg(v.key_skills, chr(9)) skills -- собираем все навыки в единую строку, в качестве разделителя используем символ табуляции
from
    vacancies v
where 
-- ограничения на сферу деятельности
(lower(v.name) like '%data scientist%' 
or lower(v.name) like '%data science%' 
or lower(v.name) like '%исследователь данных%'
or lower(v.name) like '%ml%' and v.name not like '%HTML%' and v.name not like '%QML%'
or lower(v.name) like '%machine learning%' 
or lower(v.name) like '%машинн%обучен%')
and
(v.key_skills is not null)
'''
skills_df = pd.read_sql_query(query_skills, connection)
# Преобразуем полученную таблицу в множество, удаляем дубликаты
skillset = set(skills_df['skills'][0].split(sep='\t'))
skillset

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\1600497561.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



{'.NET',
 '.NET Framework',
 '1C ERP',
 '1C: Бухгалтерия',
 '1С программирование',
 '1С-Битрикс',
 '1С: Документооборот',
 '1С: Зарплата и управление персоналом',
 '1С: Предприятие 8',
 '3D',
 '3D Data Cloud',
 'A/B тесты',
 'ABC-анализ',
 'AI',
 'AML',
 'API',
 'ARIS',
 'AWS',
 'AWS Cloud',
 'Adobe Photoshop',
 'Agile',
 'Agile Project Management',
 'AirFlow',
 'Airflow',
 'Ajax',
 'Algorithms',
 'Analyst',
 'Ansible',
 'Apache Kafka',
 'Apache Maven',
 'Atlassian Confluence',
 'Atlassian Jira',
 'Audio analytics',
 'AutoML',
 'Azure',
 'B2B Продажи',
 'BERT',
 'BI',
 'BPMN',
 'Backend',
 'Banking compliance',
 'Bash',
 'Big Data',
 'BigQuery',
 'Bitbucket',
 'Bitrix',
 'Blockchain',
 'Business Analysis',
 'Business Intelligence Systems',
 'C#',
 'C++',
 'C/C++',
 'CI/CD',
 'CJM',
 'CRISP-DM',
 'CRM',
 'CSS',
 'CSS3',
 'CUDA',
 'CV',
 'Caffe',
 'CatBoost',
 'CentOS',
 'Cisco',
 'Classic Machine Learning',
 'Classification',
 'ClickHouse',
 'Clickhouse',
 'Colvir',
 'Communication skil

In [52]:
# Создаем новый DataFrame, где будем хранить данные по частоте упоминаня навыков в описании вакансий
skill_df = pd.DataFrame()
# Обходим множество навыков из предыдущего пунка в цикле, для каждого получаем данные по частоте упоминания. 

for now in skillset: 
    query_extra = f'''
    select
        '{now}' as skill,
        count(v.id)
    from
        vacancies v
    where 
    -- сохраняем ограничения по сфере деятельности, чтобы избежать упоминания похожих навыков из других областей
    (lower(v.name) like '%data scientist%' 
    or lower(v.name) like '%data science%' 
    or lower(v.name) like '%исследователь данных%'
    or lower(v.name) like '%ml%' and v.name not like '%HTML%' and v.name not like '%QML%'
    or lower(v.name) like '%machine learning%' 
    or lower(v.name) like '%машинн%обучен%')
    and
    (v.key_skills like '%{now}%')
    '''
    # Считываем запрос в однострочный временный датафрейм 
    temp = pd.read_sql_query(query_extra, connection)
    # Добавляем полученные данные в таблицу с навыками
    skill_df = pd.concat([skill_df,temp])

C:\Users\SorkC\AppData\Local\Temp\ipykernel_4596\1989922371.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [53]:
# Выведем график топ-10 востребованных навыков:
fig = px.bar(skill_df.sort_values(by='count', ascending=False).head(10), y='count', x='skill', text_auto=True)
fig.show()

# Общий вывод по проекту

В целом из проведенного анализа можно сделать вывод о большом запросе на IT специалистов в современных условиях. При этом крупные компании нанимают большое количество сотрудников, указывают более полную информацию о компании и вакансии в описании - указаны зарплатные вилки, сверы деятельности, график и режим работы и т.п.. Очевидно, есть связь "чем крупнее город - тем больше вакакнсий".

В болке предметного анализа мы выяснили, на какие условия может рассчитывать начинающий специалсти в области обработки данных, какие существуют требования к опыту соискателя и его навыкам. А также вывели топ-10 навыков, которые работодатели хотели бы видеть в успешном кандидате. 

В дальнейшем можно развить анализ навыков, чтобы выделить самые востребованные навыки для каждого уровня опыта. Можно проверить корреляцию зарплатных предложений с регионом размещения и\или типом занятости. Можно выяснить, акие компании активно предлагают удаленный или гибридный формат работы, и в каких сферах. Также можно попробовать провести подобный анализ во времени, чтобы проверить как изменился рынок труда за последнее время. Например, насколько отличается средняя предлагаемая зарплата от средней и медианной по стране\региону в современных условиях, и какое было соотношение за последние 5 лет. 
Можно посмотреть, как изменились требования работодателей с течением времени, какие навыки набирают популярность, а значит, их изучению стоит уделить внимание. Дополнительные исследования я разместил в областях выводов соответствующих блоков для простоты и наглядности. 